<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>CS 144 - Spring 2021 - Gene Finding</h1></center>
<center><h1>Due: Sunday, June 6th, 2021 @ 11:59pm</h1></center>

### Enter your information below:

<div style="color: #000000;background-color: #EEEEFF">
    Your Name (submitter):  <br>
    Your student ID (submitter):
<br>
<br>
<b>By submitting this notebook, I assert that the work below is my own work, completed for this course.  Except where explicitly cited, none of the portions of this notebook are duplicated from anyone else's work or my own previous work.</b>
<br>    
<br>
<b>Instruction for submissions:</B> when you have completed this project, download this .ipynb file to your computer by left-clicking on the file name, and submit to <a href="https://elearn.ucr.edu/">Canvas</A> by the deadline. 
<br>
<br>
<B>Late work:</B> There is no late deadline for the final project, except for the most serious circumstances (illness, medical emergency, etc.) which have to be documented.
</div>


<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>Gene finding in the BD genome</h1></center>
<br>

The objective of this project is to find genes in a real genome (called BD) by analyzing real RNA-Seq data. The <i>BD</i> genome sequence is not available, and as far as I know, nobody knows where the genes are. <i>BD</i> is an a infectious parasite that causes a disease in human similar to Lyme disease. Both diseases are transmitted to humans through the bite of an infected tick. Finding the genes in <i>BD</i>  is crucial in our efforts to develop new drugs to prevent this parasite from harming humans.

The genome assembly of <i>BD</i> was assembled in my lab from Pacific Bioscence HiFi reads, the latest technology for DNA sequencing. We believe that <i>BD</i> has four/five chromosomes, but we are not sure yet. The assembly is currently broken up into nine contigs, some of which could be complete chromosomes but we are not sure either. The RNA-Seq reads are Pacific Bioscence IsoSeq, which is the latest third-generation sequencing technology to generate full-length cDNA sequences to characterize transcript isoforms across an entire transcriptome.

This project is somewhat open-ended, and is suitable only for students who is willing to invest more time that the other projects. Other than having fun working with a real research problem, your ideas could generate a new method in bioinformatics (i.e., this is as close to research as you will get in this class). In order to discover the genes you will have to come up with your own algorithm. You can discuss your initial idea with me, but you will responsible to justify your design choices. Your algorithm has to run in a reasonable amount of time (at least on the smallest BD chromosome) so that you can demonstrate it during the demo.

I will provide you with our best guess where the genes are, so that you compare the results on your method against our gene predictions.

First, let's get the data using wget. The file `BD.fa` is the genome (in FASTA format), while the file `transcripts.fastq` contains the transcripts (FASTQ format). Please run the code below only once or you will end up with a lot of copies of these files.

In [ ]:
!wget http://www.cs.ucr.edu/~stelo/cs144spring21/data/BD.fa
!wget http://www.cs.ucr.edu/~stelo/cs144spring21/data/transcripts.fastq

Let's take a quick look at the content of these files using `head`. You can also use the JupyterLab Launcher to open a terminal, navigate to the directory where these files are located and type `less transcripts.fastq` or `less BD.fa`. Observe that the genome is in FASTA format (read this <A href="https://en.wikipedia.org/wiki/FASTA_format">link</A> for a description) while the transcripts are in FASTQ format (read this <A href="https://en.wikipedia.org/wiki/FASTQ_format">link</A> for a description). You can use Biophyton to parse FASTA or FASTQ. Observe that the trascripts have quality scores.

In [ ]:
!head BD.fa

In [ ]:
!head transcripts.fastq

Next, we will use `minimap2` to map the reads/trascripts against the BD genome. We are using a spliced alignment for high quality reads (option `-ax splice:hq`), we are forcing `minimap2` to consider the forward transcript strand only (option `-u f`), we are using twenty CPU cores (option `-t 20`). `Minimap2` generates the index (based on the BWT) on the fly, and maps over 46 thousand transcripts in less than 5 seconds. `Minimap2` (like all the other read mappers) generates a Sequence Alignment Map (`.sam`) file, which is a text file formatted according to <A HREF="https://samtools.github.io/hts-specs/SAMv1.pdf">these specifications</A>. 

In [ ]:
!minimap2 -ax splice:hq -uf -t 20 ./BD.fa ./transcripts.fastq  > ./minimap2.sam

We can convert the SAM file `minimap2.sam` into an equivalent binary file (`.bam`) file, which is more space efficient and can be sorted by genomic position. We will be using the tool `samtools` to (1) convert from SAM to BAM, then (2) sort the BAM file, and finally (3) index the BAM file so that we can achieve efficient access. The `.sam` file can be deleted after we get the `bam` file.

In [ ]:
!samtools view -S -bT ./BD.fa ./minimap2.sam > ./minimap2.bam
!samtools sort -o ./minimap2sorted.bam ./minimap2.bam
!samtools index ./minimap2sorted.bam
!rm -f ./minimap2.sam

We will be using <A HREF="https://pysam.readthedocs.io/en/latest/index.html">PYASM</A> to process the BAM file. Let's first inspect the header files. The header is represented as a Python dictionary (where the key is the record_type). As there can be several instances of the same record_type, the value of the dictionary is a list (where each element is, again, a dictionary, or sometimes a string containing tag/value pairs). Here we are interested in capturing the name of the chromosomes/contigs (field SN), and their length (field LN).

In [ ]:
import pysam
bam = pysam.AlignmentFile('minimap2sorted.bam', 'rb')
headers = bam.header
chromosome_name = []
chromosome_length = []
for record_type, records in headers.items():
    # print (record_type) # uncomment to see the records
    for i, record in enumerate(records):
        if type(record) == dict:
            # print('\t%d' % (i + 1)) # uncomment to see the records
            for field, value in record.items():
                # print('\t\t%s\t%s' % (field, value)) # uncomment to see the records
                if (field=='SN'):
                    chromosome_name.append(value)
                if (field=='LN'):
                    chromosome_length.append(value)

#print()
print('contigs =', chromosome_name)
print('lengths (bp) =', chromosome_length)

Next we use <A HREF="https://pysam.readthedocs.io/en/latest/index.html">PYASM</A> to compute the transcript coverage across each chromosome/contig from the BAM file produced by `minimap2`. The <i>sequencing coverage</i> (or <i>sequencing depth</i>) is defined as the number of reads (transcripts in this case) that cover a particular position in the genome. Remember that genes can be on the positive strand or the negative strand of the genome, so we need to process each strand separateraly. The purpose of the next code is to compute the arrays `coveragepositive` and `coveragenegative` for the chromosome/contig specified by the variable `c`. Below we plot the positive and negative coverage for contig9 (`c=8`).

In [ ]:
%matplotlib inline
import pysam
import matplotlib.pyplot as plt

minimap2 = pysam.AlignmentFile("minimap2sorted.bam", "rb" )
c = 8 # the contig we want to process; 0 is contig1, 1 is contig2, ..., 8 is contig9

# create an empty array with zeros of size chromosome_length[c]
coveragepositive = [0] * chromosome_length[c]
coveragenegative = [0] * chromosome_length[c]

# iterate over all the positions in chromosome c
for pileupcolumn in minimap2.pileup(chromosome_name[c]):  #, 24000, 24500
    positive = 0
    negative = 0
    positive_del = 0
    negative_del = 0
    # iterate over all the mapped reads that overlap that position
    for pileupread in pileupcolumn.pileups:
        if pileupread.alignment.is_reverse: # is this read mapped on the reverse strand?
            negative += 1
            # check whether the read has a deletion or an "N" in that position, if so, count it
            if pileupread.is_del and pileupread.is_refskip:
                negative_del += 1            
        else:
            positive += 1
            # check whether the read has a deletion or an "N" in that position, if so, count it
            if pileupread.is_del and pileupread.is_refskip:
                positive_del += 1     
    # the coverage is the number of reads mapped in that position minus the number of reads that have a deletion
    coveragepositive[pileupcolumn.pos] = positive - positive_del
    coveragenegative[pileupcolumn.pos] = negative - negative_del
    
fig, ax = plt.subplots(figsize=(30,9))
ax.plot(range(0, chromosome_length[c]), coveragepositive)
fig, ax = plt.subplots(figsize=(30,9))
ax.plot(range(0, chromosome_length[c]), coveragenegative)

Let's zoom in into the region (342000,344845) at the very end (telomere) of this chromosome/contig on the negative strand. This is a very clear gene on the negative strand (see below).

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(30,9))
begin = 342000 
end = 344845
ax.plot(range(begin, end), coveragenegative[begin:end])

The other telomeric region (21000,36000) on this chromosome/contig is more complex.

In [ ]:
%matplotlib inline
begin = 21000 
end = 36000
fig, ax = plt.subplots(figsize=(30,9))
ax.plot(range(begin, end), coveragepositive[begin:end])
fig, ax = plt.subplots(figsize=(30,9))
ax.plot(range(begin, end), coveragenegative[begin:end])

There is a gene from position approx 21800 to  approx 30900 on the positive strand, while on the negative strand the gene is from approx 23900 to approx 35000.

The idea of this project is to use the coverage histogram to find all the genes in the BD genome. The genes in BD would correspond to genomic regions which are coveread by RNA-Seq reads (since these reads  correspond to mRNA transcripts). Note, however that the coverage can be pretty uneven, and finding the boundaries of the genes is not trivial. Coverage "valleys" inside a genome correspond to introns (which are not present in the transcript) and can confuse your algorithm.

The objective of this project is to come up with an algorithm to process the coverage histogram, and produce the list of predicted genes in the BD genome in all chromosomes/contigs. The output should a tab-separated text file with on gene per line with the format `gene_name`, `contig#`, `start`, `end`, and `strand`. Please note that `start` is always smaller than `end` even when the gene is on the negative strand. For example the following output

    BD.0001  contig1   20543    40105   -
    BD.0002  contig1   60043    65323   +
    BD.0003  contig3  133455   135034   +
    BD.0004  contig5   20344    34124   -
    ...
    
indicates that your method detected a gene `BD.0001` (the name of the gene is arbitrary) on contig1 that starts at position 20543 and ends at position 40105 on the negative strand. Gene `BD.0003` is on contig3, etc.

The position of the genes detected by our pipeline is next. You can use it to test or train your method. If you use it to train it, keep some of this data for testing.

In [ ]:
!wget http://www.cs.ucr.edu/~stelo/cs144spring21/data/genes.txt

In [ ]:
!head genes.txt

In [ ]:
minimap2.close()
!rm -f minimap2* BD.* transcripts* genes.txt 2>/dev/null # uncomment this to clean up your directory